# Fraud detection

This time, fraud detection was conducted using an autoencoder. Both stacked and convolutional autoencoders were used.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import seaborn as sns
import kagglehub
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report, confusion_matrix

# Download dataset





In [22]:
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
data = pd.read_csv(f"{path}/creditcard.csv")
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [23]:
data.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [24]:
data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [25]:
#X = data.drop(columns={'Class','Time'}).values
X = data[['V4','V11']].values
y = data['Class'].values

Scaler = StandardScaler()
X = Scaler.fit_transform(X)
X

array([[ 0.97336551, -0.54040704],
       [ 0.31652293,  1.58000285],
       [ 0.26823129,  0.61182971],
       ...,
       [-0.39398367,  0.40326169],
       [ 0.48719238, -1.89460912],
       [-0.35757   , -1.01934641]])

In [26]:
# Convert to NumPy and then cast to float32 type
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Create a TensorDataset
dataset = TensorDataset(X_tensor, y_tensor)

Autoencoders detect anomalies by learning the patterns of normal data and identifying deviations based on reconstruction errors. The encoder compresses the input, while the decoder reconstructs it, aiming to minimize the reconstruction error. When anomalous data is input, the reconstruction error increases as the model struggles to reconstruct patterns it has not seen before. Anomalies are identified by setting a threshold on this error. Without the decoder, reconstruction error cannot be computed, making it impossible to distinguish anomalies from normal data, which is why the decoder is essential for anomaly detection.

In this analysis, both a stacked autoencoder and a convolutional autoencoder were used, and the performance was compared using ReLU and SELU as activation functions.

# Stacked Autoencoder

In [27]:
# Convert to NumPy and then cast to float32 type
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Create a TensorDataset
dataset = TensorDataset(X_tensor, y_tensor)

In [33]:
#Data Loading and Preprocessing
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train only on normal data (e.g., select data where Class = 0)
X_train_normal = X[y == 0]  # Select only normal transaction data
X_tensor = torch.tensor(X_train_normal, dtype=torch.float32)

# Create TensorDataset and DataLoader
batch_size = 128
dataset = TensorDataset(X_tensor)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

#Define the Autoencoder Model
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 8),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(8, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Initialize the model
input_dim = X.shape[1]  # Number of features
model = Autoencoder(input_dim).to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()  # Compute reconstruction error
optimizer = optim.Adam(model.parameters(), lr=0.001)

#Train the Autoencoder
epochs = 20
for epoch in range(epochs):
    total_loss = 0
    for batch_X in train_loader:
        batch_X = batch_X[0].to(device)

        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_X)  # Compute MSE loss

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {total_loss / len(train_loader):.4f}')

print("Autoencoder training complete!")

Epoch [1/20], Loss: 0.5776
Epoch [2/20], Loss: 0.5598
Epoch [3/20], Loss: 0.5598
Epoch [4/20], Loss: 0.5597
Epoch [5/20], Loss: 0.5597
Epoch [6/20], Loss: 0.5598
Epoch [7/20], Loss: 0.5597
Epoch [8/20], Loss: 0.5596
Epoch [9/20], Loss: 0.5597
Epoch [10/20], Loss: 0.5597
Epoch [11/20], Loss: 0.5597
Epoch [12/20], Loss: 0.5597
Epoch [13/20], Loss: 0.5596
Epoch [14/20], Loss: 0.5598
Epoch [15/20], Loss: 0.5597
Epoch [16/20], Loss: 0.5597
Epoch [17/20], Loss: 0.5597
Epoch [18/20], Loss: 0.5597
Epoch [19/20], Loss: 0.5597
Epoch [20/20], Loss: 0.5596
Autoencoder training complete!


In [34]:
# Compute reconstruction error and detect fraudulent transactions
X_test_tensor = torch.tensor(X, dtype=torch.float32).to(device)  # Test on the entire dataset
y_test = y  # Actual labels

# Model prediction (compute reconstruction error)
with torch.no_grad():
    X_reconstructed = model(X_test_tensor)

# Compute MSE loss (reconstruction error)
reconstruction_error = torch.mean((X_test_tensor - X_reconstructed) ** 2, axis=1).cpu().numpy()

# Detect fraudulent transactions (set threshold)
threshold = np.percentile(reconstruction_error, 95)  # Set the top 5% as fraudulent transactions

# Prediction results (Class = 1 if the error exceeds the threshold)
y_pred = (reconstruction_error > threshold).astype(int)

# Performance evaluation
from sklearn.metrics import classification_report, confusion_matrix

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[270411  13904]
 [   155    337]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.97    284315
           1       0.02      0.68      0.05       492

    accuracy                           0.95    284807
   macro avg       0.51      0.82      0.51    284807
weighted avg       1.00      0.95      0.97    284807



## SELU

In [9]:
#Define the Autoencoder Model
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.SELU(),
            nn.Linear(64, 32),
            nn.SELU(),
            nn.Linear(32, 16),
            nn.SELU(),
            nn.Linear(16, 8),
            nn.SELU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(8, 16),
            nn.SELU(),
            nn.Linear(16, 32),
            nn.SELU(),
            nn.Linear(32, 64),
            nn.SELU(),
            nn.Linear(64, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Initialize the model
input_dim = X.shape[1]  # Number of features
model = Autoencoder(input_dim).to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()  # Compute reconstruction error
optimizer = optim.Adam(model.parameters(), lr=0.001)

#Train the Autoencoder
epochs = 20
for epoch in range(epochs):
    total_loss = 0
    for batch_X in train_loader:
        batch_X = batch_X[0].to(device)

        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_X)  # Compute MSE loss

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {total_loss / len(train_loader):.4f}')

print("Autoencoder training complete!")

Epoch [1/20], Loss: 0.8105
Epoch [2/20], Loss: 0.7507
Epoch [3/20], Loss: 0.7359
Epoch [4/20], Loss: 0.7209
Epoch [5/20], Loss: 0.7106
Epoch [6/20], Loss: 0.7048
Epoch [7/20], Loss: 0.7009
Epoch [8/20], Loss: 0.6984
Epoch [9/20], Loss: 0.6967
Epoch [10/20], Loss: 0.6956
Epoch [11/20], Loss: 0.6934
Epoch [12/20], Loss: 0.6923
Epoch [13/20], Loss: 0.6886
Epoch [14/20], Loss: 0.6859
Epoch [15/20], Loss: 0.6845
Epoch [16/20], Loss: 0.6838
Epoch [17/20], Loss: 0.6832
Epoch [18/20], Loss: 0.6822
Epoch [19/20], Loss: 0.6816
Epoch [20/20], Loss: 0.6814
Autoencoder training complete!


In [10]:
# Compute reconstruction error and detect fraudulent transactions
X_test_tensor = torch.tensor(X, dtype=torch.float32).to(device)  # Test on the entire dataset
y_test = y  # Actual labels

# Model prediction (compute reconstruction error)
with torch.no_grad():
    X_reconstructed = model(X_test_tensor)

# Compute MSE loss (reconstruction error)
reconstruction_error = torch.mean((X_test_tensor - X_reconstructed) ** 2, axis=1).cpu().numpy()

# Detect fraudulent transactions (set threshold)
threshold = np.percentile(reconstruction_error, 95)  # Set the top 5% as fraudulent transactions

# Prediction results (Class = 1 if the error exceeds the threshold)
y_pred = (reconstruction_error > threshold).astype(int)

# Performance evaluation
from sklearn.metrics import classification_report, confusion_matrix

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[270503  13812]
 [    63    429]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.97    284315
           1       0.03      0.87      0.06       492

    accuracy                           0.95    284807
   macro avg       0.51      0.91      0.52    284807
weighted avg       1.00      0.95      0.97    284807



# Convolutional autoencoder

In [11]:
#Define the Convolutional Autoencoder Model
class ConvAutoencoder(nn.Module):
    def __init__(self, input_dim):
        super(ConvAutoencoder, self).__init__()

        # Encoder (Convolutional Layers)
        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.SELU(),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.SELU(),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.SELU()
        )

        # Decoder (Transpose Convolutions to Restore Original Shape)
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.SELU(),
            nn.ConvTranspose1d(in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.SELU(),
            nn.ConvTranspose1d(in_channels=16, out_channels=1, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()  # To keep output values between 0 and 1
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Initialize the model
input_dim = X.shape[1]  # Number of features (30)
model = ConvAutoencoder(input_dim).to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#Training Loop
epochs = 20
for epoch in range(epochs):
    total_loss = 0
    for batch_X in train_loader:
        batch_X = batch_X[0].to(device)  # Extract tensor
        batch_X = batch_X.unsqueeze(1)  # Ensure shape is (batch_size, 1, 30)

        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_X)  # Compute reconstruction loss

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {total_loss / len(train_loader):.4f}')

print("Training Complete!")

Epoch [1/20], Loss: 0.6344
Epoch [2/20], Loss: 0.6326
Epoch [3/20], Loss: 0.6323
Epoch [4/20], Loss: 0.6323
Epoch [5/20], Loss: 0.6323
Epoch [6/20], Loss: 0.6321
Epoch [7/20], Loss: 0.6322
Epoch [8/20], Loss: 0.6326
Epoch [9/20], Loss: 0.6326
Epoch [10/20], Loss: 0.6321
Epoch [11/20], Loss: 0.6321
Epoch [12/20], Loss: 0.6322
Epoch [13/20], Loss: 0.6321
Epoch [14/20], Loss: 0.6321
Epoch [15/20], Loss: 0.6321
Epoch [16/20], Loss: 0.6322
Epoch [17/20], Loss: 0.6321
Epoch [18/20], Loss: 0.6322
Epoch [19/20], Loss: 0.6321
Epoch [20/20], Loss: 0.6321
Training Complete!


In [12]:
#Convert input data dimensions (compatible with Conv1D)
X_test_tensor = torch.tensor(X, dtype=torch.float32).to(device)

# Conv1D requires input in the shape: (batch_size, channels, sequence_length)
X_test_tensor = X_test_tensor.unsqueeze(1)  # Convert to shape: (batch_size, 1, features)
print("Shape after unsqueeze:", X_test_tensor.shape)

# Model prediction (compute reconstruction error)
with torch.no_grad():
    X_reconstructed = model(X_test_tensor)

#Compute MSE loss (reconstruction error) over both channel and sequence length dimensions
#This produces a 1D array with one error per sample
reconstruction_error = torch.mean((X_test_tensor - X_reconstructed) ** 2, dim=(1,2)).cpu().numpy()

#Fraud detection (set threshold)
threshold = np.percentile(reconstruction_error, 95)  # Set the top 2% as fraudulent transactions

#Prediction results: if error > threshold, predict Class=1 (fraud), else Class=0 (normal)
y_pred = (reconstruction_error > threshold).astype(int)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("Classification Report:")
print(classification_report(y_test, y_pred))

Shape after unsqueeze: torch.Size([284807, 1, 29])
Confusion Matrix:
[[270502  13813]
 [    64    428]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.97    284315
           1       0.03      0.87      0.06       492

    accuracy                           0.95    284807
   macro avg       0.51      0.91      0.52    284807
weighted avg       1.00      0.95      0.97    284807



High reconstruction error (loss) can affect the ability to differentiate between normal and fraudulent data. Although the CNN autoencoder shows slightly better performance, overall the models do not reconstruct normal data well. This makes it difficult to set an appropriate threshold for detecting fraudulent transactions, ultimately reducing the performance in distinguishing between normal and fraudulent transactions.

And when looking at the classification results, the precision and F1-score for fraud are very low. This is due to class imbalance, so to address this issue, it is necessary to mitigate the imbalance in the data through appropriate sampling methods.